# Challenge 2: Sentiment Analysis

In this challenge we will learn sentiment analysis and practice performing sentiment analysis on Twitter tweets. 

## Introduction

Sentiment analysis is to *systematically identify, extract, quantify, and study affective states and subjective information* based on texts ([reference](https://en.wikipedia.org/wiki/Sentiment_analysis)). In simple words, it's to understand whether a person is happy or unhappy in producing the piece of text. Why we (or rather, companies) care about sentiment in texts? It's because by understanding the sentiments in texts, we will be able to know if our customers are happy or unhappy about our products and services. If they are unhappy, the subsequent action is to figure out what have caused the unhappiness and make improvements.

Basic sentiment analysis only understands the *positive* or *negative* (sometimes *neutral* too) polarities of the sentiment. More advanced sentiment analysis will also consider dimensions such as agreement, subjectivity, confidence, irony, and so on. In this challenge we will conduct the basic positive vs negative sentiment analysis based on real Twitter tweets.

NLTK comes with a [sentiment analysis package](https://www.nltk.org/api/nltk.sentiment.html). This package is great for dummies to perform sentiment analysis because it requires only the textual data to make predictions. For example:

```python
>>> from nltk.sentiment.vader import SentimentIntensityAnalyzer
>>> txt = "Ironhack is a Global Tech School ranked num 2 worldwide.   Our mission is to help people transform their careers and join a thriving community of tech professionals that love what they do."
>>> analyzer = SentimentIntensityAnalyzer()
>>> analyzer.polarity_scores(txt)
{'neg': 0.0, 'neu': 0.741, 'pos': 0.259, 'compound': 0.8442}
```

In this challenge, however, you will not use NLTK's sentiment analysis package because in your Machine Learning training in the past 2 weeks you have learned how to make predictions more accurate than that. The [tweets data](https://www.kaggle.com/kazanova/sentiment140) we will be using today are already coded for the positive/negative sentiment. You will be able to use the Naïve Bayes classifier you learned in the lesson to predict the sentiment of tweets based on the labels.

## Conducting Sentiment Analysis

### Loading and Exploring Data

The dataset we'll be using today is located in the lab directory named `Sentiment140.csv.zip`. You need to unzip it into a `.csv` file. Then in the cell below, load and explore the data.

*Notes:* 

* The dataset was downloaded from [Kaggle](https://www.kaggle.com/kazanova/sentiment140). We made a slight change on the original data so that each column has a label.

* The dataset is huuuuge (1.6m tweets). When you develop your data analysis codes, you can sample a subset of the data (e.g. 20k records) so that you will save a lot of time when you test your codes.

In [29]:
import pandas as pd
import nltk
from nltk.tokenize import word_tokenize
from nltk.stem import PorterStemmer 
from nltk.stem.snowball import SnowballStemmer
from nltk.stem import WordNetLemmatizer 
from nltk.corpus import stopwords
from textblob import TextBlob
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.feature_extraction.text import TfidfVectorizer

In [30]:
sent = pd.read_csv('../../../../Downloads/Sentiment140.csv')
sent_sampled = sent.sample(20000)

### Prepare Textual Data for Sentiment Analysis

Now, apply the functions you have written in Challenge 1 to your whole data set. These functions include:

* `clean_up()`

* `tokenize()`

* `stem_and_lemmatize()`

* `remove_stopwords()`

Create a new column called `text_processed` in the dataframe to contain the processed data. At the end, your `text_processed` column should contain lists of word tokens that are cleaned up. Your data should look like below:

![Processed Data](data-cleaning-results.png)

In [31]:
import re

def clean_up(s):
    """
    Cleans up numbers, URLs, and special characters from a string.

    Args:
        s: The string to be cleaned up.

    Returns:
        A string that has been cleaned up.
    """
    s=re.sub('http://.*', ' ', s).lower()
    s=re.sub('[^A-Za-z]+', ' ', s)
    
    return s

In [32]:
def tokenize(s):
    """
    Tokenize a string.

    Args:
        s: String to be tokenized.

    Returns:
        A list of words as the result of tokenization.
    """
    return word_tokenize(s)


In [33]:
def stem_and_lemmatize(l):
    """
    Perform stemming and lemmatization on a list of words.

    Args:
        l: A list of strings.

    Returns:
        A list of strings after being stemmed and lemmatized.
    """
    ps = PorterStemmer ()
    stemmed = [ps.stem(w) for w in l]
    lemmatizer = WordNetLemmatizer()
    lemmatized = [lemmatizer.lemmatize(w) for w in stemmed]
    return lemmatized

In [34]:
def remove_stopwords(l):
    """
    Remove English stopwords from a list of strings.

    Args:
        l: A list of strings.

    Returns:
        A list of strings after stop words are removed.
    """
    
    stop_words = stopwords.words('english')
    
    return [w for w in l if w not in stop_words]

In [35]:
sent_sampled['text_processed'] = sent_sampled.text.apply(clean_up)
sent_sampled['text_processed'] = sent_sampled.text_processed.apply(tokenize)
sent_sampled['text_processed'] = sent_sampled.text_processed.apply(stem_and_lemmatize)
sent_sampled['text_processed'] = sent_sampled.text_processed.apply(remove_stopwords)
sent_sampled

,target,id,date,flag,user,text,text_processed
1216584,4,1989656020,Mon Jun 01 02:20:04 PDT 2009,NO_QUERY,joelyme3D,hmm might have a nice long shower or bath to h...,"[hmm, might, nice, long, shower, bath, help, r..."
1322554,4,2014804972,Wed Jun 03 02:44:44 PDT 2009,NO_QUERY,_iDANCE19,"For me, it's goodnight xo P.S. Be sure to re...","[goodnight, xo, p, sure, read, love, never, fail]"
1427990,4,2059480879,Sat Jun 06 17:01:17 PDT 2009,NO_QUERY,UnveilingHope,@stephfortunato Hope you put sunscreen on that...,"[stephfortunato, hope, put, sunscreen, pretti,..."
626179,0,2230505832,Thu Jun 18 17:12:31 PDT 2009,NO_QUERY,OnePersonToo,Wondering why Tweetie did not update my icon.,"[wonder, whi, tweeti, updat, icon]"
1518459,4,2175965203,Mon Jun 15 02:31:07 PDT 2009,NO_QUERY,drivenrajat,@reemakoul @purugovind public forum hai.. naam...,"[reemakoul, purugovind, public, forum, hai, na..."
...,...,...,...,...,...,...,...
1040378,4,1956940374,Thu May 28 23:05:01 PDT 2009,NO_QUERY,Peeeeches,@kinkyb24 I didn't see ur tweet until now. Sor...,"[kinkyb, see, ur, tweet, sorri, love, u]"
756231,0,2288656174,Mon Jun 22 19:26:48 PDT 2009,NO_QUERY,bubbleMAMI,Run's family makes me happy. Thats a frown be...,"[run, famili, make, happi, frown, becaus, expr..."
745623,0,2282483738,Mon Jun 22 11:25:37 PDT 2009,NO_QUERY,Superpaperlink,Wishes theresa had not come to shops with me a...,"[wish, theresa, come, shop, charlott, would, a..."
1091246,4,1969872534,Sat May 30 01:32:59 PDT 2009,NO_QUERY,nickash20,Kickback fun!!!! Time to get buckwild HaHahaHa...,"[kickback, fun, time, get, buckwild, hahahaha,..."


### Creating Bag of Words

The purpose of this step is to create a [bag of words](https://en.wikipedia.org/wiki/Bag-of-words_model) from the processed data. The bag of words contains all the unique words in your whole text body (a.k.a. *corpus*) with the number of occurrence of each word. It will allow you to understand which words are the most important features across the whole corpus.

Also, you can imagine you will have a massive set of words. The less important words (i.e. those of very low number of occurrence) do not contribute much to the sentiment. Therefore, you only need to use the most important words to build your feature set in the next step. In our case, we will use the top 5,000 words with the highest frequency to build the features.

In the cell below, combine all the words in `text_processed` and calculate the frequency distribution of all words. A convenient library to calculate the term frequency distribution is NLTK's `FreqDist` class ([documentation](https://www.nltk.org/api/nltk.html#module-nltk.probability)). Then select the top 5,000 words from the frequency distribution.

In [36]:
words = [word for line in sent_sampled.text_processed for word in line]
fdist = nltk.FreqDist(words)
top_words = [tuple[0] for tuple in fdist.most_common(5000)]

### Building Features

Now let's build the features. Using the top 5,000 words, create a 2-dimensional matrix to record whether each of those words is contained in each document (tweet). Then you also have an output column to indicate whether the sentiment in each tweet is positive. For example, assuming your bag of words has 5 items (`['one', 'two', 'three', 'four', 'five']`) out of 4 documents (`['A', 'B', 'C', 'D']`), your feature set is essentially:

| Doc | one | two | three | four | five | is_positive |
|---|---|---|---|---|---|---|
| A | True | False | False | True | False | True |
| B | False | False | False | True | True | False |
| C | False | True | False | False | False | True |
| D | True | False | False | False | True | False|

However, because the `nltk.NaiveBayesClassifier.train` class we will use in the next step does not work with Pandas dataframe, the structure of your feature set should be converted to the Python list looking like below:

```python
[
	({
		'one': True,
		'two': False,
		'three': False,
		'four': True,
		'five': False
	}, True),
	({
		'one': False,
		'two': False,
		'three': False,
		'four': True,
		'five': True
	}, False),
	({
		'one': False,
		'two': True,
		'three': False,
		'four': False,
		'five': False
	}, True),
	({
		'one': True,
		'two': False,
		'three': False,
		'four': False,
		'five': True
	}, False)
]
```

To help you in this step, watch the [following video](https://www.youtube.com/watch?v=-vVskDsHcVc) to learn how to build the feature set with Python and NLTK. The source code in this video can be found [here](https://pythonprogramming.net/words-as-features-nltk-tutorial/).

[![Building Features](building-features.jpg)](https://www.youtube.com/watch?v=-vVskDsHcVc)

In [37]:
def find_features(text):
    words = text.lower()
    features = {}
    for w in top_words: features[w] = w in words
    return features

featuresets = [(find_features(t),bool(s)) for t,s in zip(sent_sampled['text'],sent_sampled['target'])]

### Building and Traininng Naive Bayes Model

In this step you will split your feature set into a training and a test set. Then you will create a Bayes classifier instance using `nltk.NaiveBayesClassifier.train` ([example](https://www.nltk.org/book/ch06.html)) to train with the training dataset.

After training the model, call `classifier.show_most_informative_features()` to inspect the most important features. The output will look like:

```
Most Informative Features
	    snow = True            False : True   =     34.3 : 1.0
	  easter = True            False : True   =     26.2 : 1.0
	 headach = True            False : True   =     20.9 : 1.0
	    argh = True            False : True   =     17.6 : 1.0
	unfortun = True            False : True   =     16.9 : 1.0
	    jona = True             True : False  =     16.2 : 1.0
	     ach = True            False : True   =     14.9 : 1.0
	     sad = True            False : True   =     13.0 : 1.0
	  parent = True            False : True   =     12.9 : 1.0
	  spring = True            False : True   =     12.7 : 1.0
```

The [following video](https://www.youtube.com/watch?v=rISOsUaTrO4) will help you complete this step. The source code in this video can be found [here](https://pythonprogramming.net/naive-bayes-classifier-nltk-tutorial/).

[![Building and Training NB](nb-model-building.jpg)](https://www.youtube.com/watch?v=rISOsUaTrO4)

In [38]:
testing_set = featuresets[:4000]
training_set = featuresets[4000:]

classifier = nltk.NaiveBayesClassifier.train(training_set)

In [39]:
classifier.show_most_informative_features(15)

Most Informative Features
                 stomach = True            False : True   =     18.8 : 1.0
                   upset = True            False : True   =     16.3 : 1.0
               congratul = True             True : False  =     13.6 : 1.0
                 headach = True            False : True   =     13.3 : 1.0
                    ughh = True            False : True   =     13.1 : 1.0
                    poor = True            False : True   =     12.7 : 1.0
                    hurt = True            False : True   =     12.4 : 1.0
                   fever = True            False : True   =     11.8 : 1.0
                  bummer = True            False : True   =     11.8 : 1.0
                    booo = True            False : True   =     11.3 : 1.0
                    argh = True            False : True   =     11.2 : 1.0
                     sad = True            False : True   =     10.8 : 1.0
                  missin = True            False : True   =     10.5 : 1.0

### Testing Naive Bayes Model

Now we'll test our classifier with the test dataset. This is done by calling `nltk.classify.accuracy(classifier, test)`.

As mentioned in one of the tutorial videos, a Naive Bayes model is considered OK if your accuracy score is over 0.6. If your accuracy score is over 0.7, you've done a great job!

In [ ]:
print("Classifier accuracy percent:", nltk.classify.accuracy(classifier, testing_set)*100)

## Bonus Question 1: Improve Model Performance

If you are still not exhausted so far and want to dig deeper, try to improve your classifier performance. There are many aspects you can dig into, for example:

* Improve stemming and lemmatization. Inspect your bag of words and the most important features. Are there any words you should furuther remove from analysis? You can append these words to further remove to the stop words list.

* Remember we only used the top 5,000 features to build model? Try using different numbers of top features. The bottom line is to use as few features as you can without compromising your model performance. The fewer features you select into your model, the faster your model is trained. Then you can use a larger sample size to improve your model accuracy score.

In [ ]:
# your code here

## Bonus Question 2: Machine Learning Pipeline

In a new Jupyter Notebook, combine all your codes into a function (or a class). Your new function will execute the complete machine learning pipeline job by receiving the dataset location and output the classifier. This will allow you to use your function to predict the sentiment of any tweet in real time. 

In [ ]:
# your code here

## Bonus Question 3: Apache Spark

If you have completed the Apache Spark advanced topic lab, what you can do is to migrate your pipeline from local to a Databricks Notebook. Share your notebook with your instructor and classmates to show off your achievements!